# Test case with Image Processing

## Baseline

Import supplyment library and function

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from image_function import *

Get file path

In [2]:
root = 'Dataset/DAPI/4x4'
files = os.listdir(root)
files = files.sort()
if '.DS_Store' in files: # for macOS file system
    files.remove('.DS_Store')
for file in files:
    file = root + '/' + file
print(files[0:3])
print(len(files))

TypeError: argument of type 'NoneType' is not iterable